In [1]:
#write function that takes query input and returns recommendation list

In [12]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import tree
import random

In [13]:
import collections

In [14]:
#load data
cols=pd.read_csv("columns.csv")
resp=pd.read_csv("responses.csv")
podcasts=pd.read_csv("df_popular_podcasts.csv")
movies=pd.read_csv("MovieGenre.csv")
music=pd.read_csv("SpotifyFeatures.csv")
genre_IDs=[['1311', 'News & Politics'], ['26', 'Podcasts'], ['1479', 'Social Sciences'], ['1315', 'Science & Medicine'], ['1324', 'Society & Culture'], ['1302', 'Personal Journals'], ['1469', 'Language Courses'], ['1304', 'Education'], ['1320', 'Places & Travel'], ['1416', 'Higher Education'], ['1465', 'Professional'], ['1316', 'Sports & Recreation'], ['1303', 'Comedy'], ['1305', 'Kids & Family'], ['1439', 'Christianity'], ['1314', 'Religion & Spirituality'], ['1444', 'Spirituality'], ['1309', 'TV & Film'], ['1462', 'History'], ['1310', 'Music'], ['1478', 'Medicine'], ['1321', 'Business'], ['1412', 'Investing'], ['1420', 'Self-Help'], ['1307', 'Health'], ['1481', 'Alternative Health'], ['1417', 'Fitness & Nutrition'], ['1467', 'Amateur'], ['1480', 'Software How-To'], ['1318', 'Technology'], ['1448', 'Tech News'], ['1456', 'Outdoor'], ['1477', 'Natural Sciences'], ['1301', 'Arts'], ['1454', 'Automotive'], ['1323', 'Games & Hobbies'], ['1438', 'Buddhism'], ['1443', 'Philosophy'], ['1401', 'Literature'], ['1402', 'Design'], ['1410', 'Careers'], ['1470', 'Training'], ['1413', 'Management & Marketing'], ['1306', 'Food'], ['1406', 'Visual Arts'], ['1446', 'Gadgets'], ['1468', 'Educational Technology'], ['1405', 'Performing Arts'], ['1460', 'Hobbies'], ['1471', 'Business News'], ['1404', 'Video Games'], ['1450', 'Podcasting'], ['1473', 'National'], ['1325', 'Government & Organizations'], ['1461', 'Other Games'], ['1466', 'College & High School'], ['1459', 'Fashion & Beauty'], ['1476', 'Non-Profit'], ['1415', 'K-12'], ['1455', 'Aviation'], ['1464', 'Other'], ['1421', 'Sexuality'], ['1472', 'Shopping'], ['1475', 'Local'], ['1441', 'Judaism'], ['1440', 'Islam'], ['1474', 'Regional'], ['1463', 'Hinduism']]

In [6]:
genre_to_music={}

for i in range(len(music)):
    genre=music['genre'][i]
    genre=genre.lower()
    if genre in genre_to_music:
            genre_to_music[genre].append(music['track_name'][i]+" by "+music['artist_name'][i])
    else:
            genre_to_music[genre]=[music['track_name'][i]+ " by "+music['artist_name'][i]]

In [ ]:
genre_to_music.keys()

In [16]:
podcast=[(i[0], i[1], list(podcasts.loc[podcasts["Name"] == i[0]]["Podcast URL"].to_dict().values())[0]) for i in  podcasts]

IndexError: list index out of range

In [ ]:
# Better organize the id vs name of genre
dict_pod={int(id[0]):id[1] for id in genre_IDs }

# {Genre ID:NAMES OF PODCAST} 
genre_to_podcast={key:[] for key in dict_pod }
for item in podcasts.index:
    array=podcasts["Genre IDs"][item][1:-1].split(',')
    array=[int(array[i].replace('\'', '').strip('\'')) for i in range(len(array))]
    for j in range(len(array)):
        genre_to_podcast[array[j]].append(podcasts["Name"][item])
#final_dict={key.lower():value for key,value in final_dict}
final_dict={list(dict_pod.values())[p]:list(genre_to_podcast.values())[p] for p in range(len(genre_to_podcast.keys()))}
final_dict = dict((k.lower(), v) for k, v in final_dict.items()) 

In [ ]:
genre_to_movie={}
for i in range( len(movies)):
    line=str(movies["Genre"][i])
    for genre in line.split("|"):
        genre=genre.lower()
        if genre in genre_to_movie:
            genre_to_movie[genre].append(movies["Title"][i])
        else: 
            genre_to_movie[genre]=[movies["Title"][i]]


In [ ]:
print(genre_to_movie)

In [ ]:
movies.columns

In [ ]:
#dependent vbs
music_cols=cols[:19]["short"]
movie_cols=cols[19:31]["short"]
hobbies=cols[31:63]["short"]

#independent vbs
personality=cols[76:133]["short"]
demographics=cols.iloc[[140, 144, 146]]["short"]

In [ ]:
def gender_numeric(gender):   
    return {
        'female': 1,
        'male':0, 
    }.get(gender,float("nan"))

resp["Gender"]=list(map(gender_numeric, list(resp["Gender"])))
resp

In [ ]:
# Convert education to quantitative values and reassign to table
def edu_level_numeric(edu):   
    return {
        'currently a primary school pupil': 0,
        'primary school': 1,
        'secondary school': 2, 
        'college/bachelor degree': 3,
        'masters degree': 4,   
        'doctorate degree': 5
    }.get(edu,float("nan"))

resp["Education"]=list(map(edu_level_numeric, list(resp["Education"])))

In [ ]:
music_resp=resp.iloc[:, :19]
movie_resp=resp.iloc[:, 19:31]
hobbies_resp=resp.iloc[:, 31:63]
personality_resp=resp.iloc[:, [79, 80, 105, 106, 109, 110, 113, 129, 132]]
demographics_resp=resp.iloc[:, [140, 144, 146, 147]]

final_mat=pd.concat([music_resp, movie_resp, hobbies_resp, personality_resp, demographics_resp], axis=1)
final_mat=final_mat.rename(columns={"New environment": "adapt", "Socializing": "meeting-people", "Waiting":"patient", "Number of friends": "friends", "Workaholism": "study", "Thinking ahead": "perspectives", "Charity": "charity", "Interests or hobbies": "differ-hobbies"})

corr_mat=pd.DataFrame.corr(final_mat)
corr_mat.columns = map(str.lower, corr_mat.columns)
corr_mat.index = map(str.lower, corr_mat.index)
corr_mat.columns

In [ ]:
def recs(genre_dict, genre_query, corr_query):
    """Returns a list of recommendations based on interests user clicked in form
    Params: {query: list of genre names, genre_dict: dictionary that maps genre to titles, corr_dict: maps personality questions to answers given in survey}
    Returns: list of tuples containing titles and scores
    """
    
    #query is an array of genre_names
    #corr_query is a dictionary of radio button responses like {'all-music': 'R&B', 'all-movies': 'Horror', 'all-interests': 'Social Sciences', 'gender': 'female', 'education': 'no'}
    if 'all-interests' in corr_query: 
        corr_query.pop('all-interests')
    if 'all-movies' in corr_query:
        corr_query.pop('all-movies')
    if 'all-music' in corr_query:
        corr_query.pop('all-music')
    if not corr_query: #check if empty
        corr_query={'charity': '3', 'adapt': '3', 'meeting-people': '3', 'patient': '3', 'friends': '3', 'study': '3', 'perspectives': '3', 'differ-hobbies': '3' }
        
    genre_query=[genre.lower() for genre in genre_query]
    
    counter={}
    for key in corr_query:
        val=float(corr_query[key]) 
        if val > 3.0:
            val = val - 3
        elif val == 3.0:
            val = 1
        elif val < 3.0:
            val = val* - 1
        for cat in genre_query: 
            corr=float(corr_mat[key][cat])
            weighted=corr*val
            for film in genre_dict[cat]:
                if film in counter:
                    counter[film]+=weighted
                else: 
                    counter[film]=weighted

    results = list(counter.items())
    random.shuffle(results)
    results.sort(key=lambda x: x[1], reverse=True)
    highest_score=results[0][1]
   
    output=[]
    for result in results[:5]:
        output.append((result[0], result[1]/highest_score))
     
    return output

In [ ]:
recs(final_dict, ['Religion & Spirituality'], {})

In [ ]:
# Convert education to quantitative values and reassign to table
def edu_level_numeric(edu):   
    return {
        'currently a primary school pupil': 0,
        'primary school': 1,
        'secondary school': 2, 
        'college/bachelor degree': 3,
        'masters degree': 4,   
        'doctorate degree': 5
    }.get(edu,float("nan"))
resp["Education"]=list(map(edu_level_numeric, list(resp["Education"])))

In [ ]:
def gender_numeric(gender):   
    return {
        'female': 0,
        'male':1
    }.get(gender,float("nan"))
resp["Gender"]=list(map(gender_numeric, list(resp["Gender"])))

In [ ]:
# NOW FOR PERSONALITY
def punct_numeric(punct):   
    return {
        'i am often early': 1,
        'i am always on time':2,
        'i am often running late': 3
    }.get(punct,float("nan"))
resp["Punctuality"]=list(map(punct_numeric, list(resp["Punctuality"])))



def ly_numeric(lying):
    return {
        'never':0,
        'only to avoid hurting someone': 1,
        'sometimes': 2,
        'everytime it suits me': 3
    }.get(lying,float("nan"))
resp["Lying"]=list(map(ly_numeric,list(resp["Lying"])))


def web_numeric(web):
    return{
        'no time at all': 0, 
        'less than an hour a day': 1,
        'few hours a day': 2,
         'most of the day': 3
    }.get(web,float("nan"))
resp["Internet usage"]=list(map(web_numeric,list(resp["Internet usage"])))

In [ ]:
corr_dict={}
for dem in demographics:
    dem_var=resp[dem]
    for col in music:
        music_var=resp[col]
        corr_dict[(col,dem)]=pd.concat([music_var,dem_var], axis=1).corr("pearson").iloc[0,1]
print("Correlations for Music and Demographic were between" , 
      sorted(corr_dict.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict.items(), key=lambda x: x[1])[-1])

n_corr_dict={}
for per in personality:
    per_var=resp[per]
    for col in music:
        music_var=resp[col]
        n_corr_dict[(col,per)]=pd.concat([music_var,per_var], axis=1).corr("pearson").iloc[0,1]

print("Correlations for Music and Personality were between" , 
      sorted(n_corr_dict.items(), key=lambda x: x[1])[0], "and",
      sorted(n_corr_dict.items(), key=lambda x: x[1])[-1])

In [ ]:
# Movies
corr_dict1={}
for dem in demographics:
    dem_var=resp[dem]
    for col in movie_cols:
        movie_var=resp[col]
        corr_dict1[(col,dem)]=pd.concat([movie_var,dem_var], axis=1).corr("pearson").iloc[0,1]

print("Correlations for Movies and Demographics were between" , 
      sorted(corr_dict1.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict1.items(), key=lambda x: x[1])[-1])




corr_dict2={}
for per in personality:
    per_var=resp[per]
    for col in movie_cols:
        movie_var=resp[col]
        corr_dict2[(col,per)]=pd.concat([movie_var,per_var], axis=1).corr("pearson").iloc[0,1]

print("Correlations for Movies and Personality were between" , 
      sorted(corr_dict2.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict2.items(), key=lambda x: x[1])[-1])

In [ ]:
# Hobbies
corr_dict3={}
for dem in demographics:
    dem_var=resp[dem]
    for col in hobbies:
        hob=resp[col]
        corr_dict3[(col,dem)]=pd.concat([hob,dem_var], axis=1).corr("pearson").iloc[0,1]
        

print("Correlations for Hobbies and Demographics were between" , 
      sorted(corr_dict3.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict3.items(), key=lambda x: x[1])[-1])



corr_dict4={}
for per in personality:
    per_var=resp[per]
    for col in hobbies:
        hob=resp[col]
        corr_dict4[(col,per)]=pd.concat([hob,per_var], axis=1).corr("pearson").iloc[0,1]
        
print("Correlations for Hobbies and Personality were between" , 
      sorted(corr_dict4.items(), key=lambda x: x[1])[0], "and",
      sorted(corr_dict4.items(), key=lambda x: x[1])[-1])
